   
   #### Canny (or similar) edge detection
    Describe the parameter values and their impact on the result. Select what you think is a set of good parameter values, apply, show and decribe the result.

    The Canny edge detector consists of multiple steps: 
    - Smoothing with a Gaussian kernel
    - Calculating the gradient magnitudes
    - Non-maximum suppression 
    - Hysterisis thresholding 

    The Gaussian smoothing kernel reduces noise, and has already been described previously. After an image has been smoothed, the Sobel operator is applied to find the gradients in the picture. Its gradient magnitudes are found by taking the derivative of the x- and y-gradients. As shown in the section "Gradient magnitude computation using Gaussian derivatives", resulting edges can span multiple pixels. These edges can be reduced to span a single pixel with non-maximum suppression. If the gradient of a pixel is greater than the two pixel gradients in its gradient direction, its value is kept, otherwise it is set to 0. Thereby, the only the largest gradient across an edge is kept. In order to find edges that are connected, the hysterisis thresholding is applied. Here, all pixel intensities below the lower threshold are discarded as edges. All intensities above the upper threshold are kept. Then only pixels that are connected to high threshold pixel intensities are kept (Canny, 1986; Forsyth & Ponce, 2011). 

    In sum, multiple parameters affect the resulting number and type of contours extracted with the edge detection. Firstly, we can set the scale value in the Gaussian kernel differently and achieve more or less blurry images as shown above. Secondly, the chosen low and high threshold in hysterisis threshold determines which edgelines are discarded. Below we test both different scale and threshold values, and their interaction. 
    
    
    Canny, J. (1986). A computational approach to edge detection. IEEE Transactions on pattern analysis and machine intelligence, (6), 679-698.
    
    Forsyth, D., & Ponce, J. (2011). Computer vision: A modern approach (p. 792). Prentice hall.

In [3]:
#!pip install opencv-python
import skimage
from skimage.io import imread
from cv2 import Canny
from scipy import ndimage
import numpy as np
import matplotlib.pyplot as plt
from skimage import img_as_float
lenna = imread("lenna.jpg").astype("float")


In [ ]:
# Scale and threshold values
scales = [1, 2, 4, 8]
low = np.percentile(cv2lenna, 5)
high = np.percentile(cv2lenna, 65)

# Define axes properties
fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(10,10))
ax_list = axes.flatten()

# Going through axes and scale values
for ax, scale in zip(ax_list, scales):
    # Smoothing
    smoothed = ndimage.gaussian_filter(cv2lenna,
                        scale, # standard deviation for Gaussian kernel
                        order=0, # An order of 0 corresponds to convolution with a Gaussian kernel. A positive order corresponds to convolution with that derivative of a Gaussian.
                        output=None, # The array in which to place the output, or the dtype of the returned array. By default an array of the same dtype as input will be created.
                        mode='reflect', # ‘reflect’, ‘constant’, ‘nearest’, ‘mirror’, ‘wrap’
                        cval=0.0, # Value to fill past edges of input if mode is ‘constant’. Default is 0.0.
                        truncate=4.0) # Truncate the filter at this many standard deviations. Default is 4.0.
    canny = Canny(smoothed, low, high) 
    ax.imshow(canny);
    ax.set_title(f"Scale: {scale}")

plt.show()
    

    It is clear that, as scale increases and more pixels are influencing the output, fewer edges are detected using the Canny operator. In other words, only the coarser and stronger edge features are detected when scale is high while finer features and less strong edges are found with lower scale values. The scale seems highly influential on the number of edges detected and too large scales result in no edges detected. 

    Next, we experiment with the values in the hysterisis thresholds. We define different threshold values using percentile values of the pixel intensity distribution in the image. We test the combination between low threshold values of 17, 30 and 59 and high threshold values of 128, 149 and 202

In [ ]:
# Create a vector from image pixels
flattened = cv2lenna.flatten()

# Creating figure
fig, ax = plt.subplots(figsize = (6,4))

# Defining low and high threshold values by their percentiles
low_thresh = np.array([np.percentile(cv2lenna, 5),np.percentile(cv2lenna, 15),np.percentile(cv2lenna, 25)])
high_thresh = np.array([np.percentile(cv2lenna, 65),np.percentile(cv2lenna, 80),np.percentile(cv2lenna, 95)])
perc = np.array([5,15,25,65,80,95])

# Histogram of pixel intensities
plt.hist(flattened, alpha = 0.5, bins = 30)

# Take every percentile value, and its percentile number
for i, p in zip(np.concatenate((low_thresh, high_thresh), axis=None), perc):
    # Adding vertical line 
    ax.axvline(i, linestyle = "-", color = "Forestgreen") 
    # Adding text
    ax.text(i+2, # To create distance to vline
            1, # Y value
            s = f" {p}th percentile: {int(i)}", # Text
            rotation=90, # Rotation
            va='bottom') # Position of text

plt.title("Percentiles")
plt.show()

In [ ]:
cv2lenna = cv2.imread("lenna.jpg")

# Constant scale value
scale = 2

# Smoothing using the Gaussian kernel
smoothed = ndimage.gaussian_filter(cv2lenna,
                        scale, # standard deviation for Gaussian kernel
                        order=0, # An order of 0 corresponds to convolution with a Gaussian kernel. A positive order corresponds to convolution with that derivative of a Gaussian.
                        output=None, # The array in which to place the output, or the dtype of the returned array. By default an array of the same dtype as input will be created.
                        mode='reflect', # ‘reflect’, ‘constant’, ‘nearest’, ‘mirror’, ‘wrap’
                        cval=0.0, # Value to fill past edges of input if mode is ‘constant’. Default is 0.0.
                        truncate=4.0) # Truncate the filter at this many standard deviations. Default is 4.0.


# Defining the thresholds in the hysterisis
low_thresh = np.array([5,15,25])
high_thresh = np.array([65, 80, 95])

# Iterating through combinations of thresholds
combi = []
for i in low_thresh:
    for j in high_thresh:
        combi.append([i,j])
        
# Figure properties
fig, axes = plt.subplots(nrows=len(low_thresh), ncols=len(high_thresh), figsize=(10,10))
ax_list = axes.flatten()

# Hysterisis thresholding   
for ax, thresholds in zip(ax_list, combi):
    canny = Canny(smoothed, np.percentile(cv2lenna, thresholds[0]), np.percentile(cv2lenna, thresholds[1]))
    ax.imshow(canny)
    ax.set_title(f"Threshold: lower = {thresholds[0]}, upper = {thresholds[1]}", fontsize = 10)
    

    In general, most edges contain a high pixel value (Canny, 1986), which could be the reason why few changes apply with low thresholds at 5th and 15th percentile. More severe changes are seen with higher upper thresholds; while many of the same contours are kept at 65th and 80th percentile value, an upper threshold at the 95th results in few but essential contours containing the profile of the object but much less detail.

    The final image will be processed with a scale value of 2, a low edge threshold at the 15th percentile, i.e. at a value of 30, and a high edge threshold at the 80th percentile, i.e. at 149.
    Some code are inspired from my exam in Visual Analytics (https://github.com/marmor97/cds-visual-exam/tree/main/src/1).

In [ ]:
scale = 2
low = np.percentile(cv2lenna, 15)
high = np.percentile(cv2lenna, 80)

fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(10,10))


smoothed = ndimage.gaussian_filter(cv2lenna,
                    scale, # standard deviation for Gaussian kernel
                    order=0, # An order of 0 corresponds to convolution with a Gaussian kernel. A positive order corresponds to convolution with that derivative of a Gaussian.
                    output=None, # The array in which to place the output, or the dtype of the returned array. By default an array of the same dtype as input will be created.
                    mode='reflect', # ‘reflect’, ‘constant’, ‘nearest’, ‘mirror’, ‘wrap’
                    cval=0.0, # Value to fill past edges of input if mode is ‘constant’. Default is 0.0.
                    truncate=4.0) # Truncate the filter at this many standard deviations. Default is 4.0.

canny = Canny(smoothed, low, high) 
axes[0].imshow(canny)

# And on top of the original image
# Defining the contours in the image
(contours,_) = cv2.findContours(canny.copy(), # using np function to make a copy rather than destroying the image itself
                 cv2.RETR_EXTERNAL, 
                 cv2.CHAIN_APPROX_SIMPLE) 

drawn = cv2.drawContours(
                 cv2lenna.copy(), # image, contours, fill, color, thickness
                 contours,
                 -1, # whihch contours to draw. -1 will draw contour for every contour that it finds
                 (255,0,0), # contour color
                 1) # Thickness
axes[1].imshow(drawn)


plt.show()

